In [ ]:
%pip install geopandas adjustText

In [1]:

from datetime import datetime

import geopandas as gp
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from shapely.geometry import Point
from adjustText import adjust_text

In [2]:

from datetime import datetime

import geopandas as gp
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from shapely.geometry import Point
from adjustText import adjust_text

# Define the start of a quarter
def quarter_start(year: int, q: int) -> datetime:
    if not 1 <= q <= 4:
        raise ValueError("Quarter must be within [1, 2, 3, 4]")

    month = [1, 4, 7, 10]
    return datetime(year, month[q - 1], 1)

# Generate the Ookla Open Data tile URL
def get_tile_url(service_type: str, year: int, q: int) -> str:
    dt = quarter_start(year, q)
    base_url = "https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance"
    url = f"{base_url}/type%3D{service_type}/year%3D{dt:%Y}/quarter%3D{q}/{dt:%Y-%m-%d}_performance_{service_type}_tiles.zip"
    return url





In [3]:
# Fetch Ookla Open Data tiles
tile_url = get_tile_url("fixed", 2023, 3)
tiles = gp.read_file(tile_url)


# Load global country boundaries dataset
country_url = "https://datahub.io/core/geo-countries/r/countries.geojson"
countries = gp.read_file(country_url)

# Filter for Egypt (ISO_A3: "EGY") and Iraq (ISO_A3: "IRQ")
target_countries = countries[countries['ISO_A3'].isin(['EGY', 'IRQ'])].to_crs(4326)
Egypt = target_countries[target_countries['ISO_A3'] == 'EGY'].to_crs(4326)
Iraq = target_countries[target_countries['ISO_A3'] == 'IRQ'].to_crs(4326)
# Load U.S. states dataset (TIGER/LINE or equivalent)
us_states_url = "https://www2.census.gov/geo/tiger/TIGER2019/STATE/tl_2019_us_state.zip"
us_states = gp.read_file(us_states_url)

# Filter for California (STATEFP: "06")
california = us_states[us_states['STATEFP'] == '06'].to_crs(4326)


In [4]:
print(target_countries.columns)

Index(['ADMIN', 'ISO_A3', 'ISO_A2', 'geometry'], dtype='object')


In [5]:
print(california.columns)

Index(['REGION', 'DIVISION', 'STATEFP', 'STATENS', 'GEOID', 'STUSPS', 'NAME',
       'LSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON',
       'geometry'],
      dtype='object')


In [6]:
target_countries["geometry"].unique()

<GeometryArray>
[<MULTIPOLYGON (((34.003 26.732, 34.004 26.708, 33.99 26.72, 33.983 26.734, 3...>, <MULTIPOLYGON (((42.897 37.325, 42.937 37.32, 42.98 37.332, 43.005 37.347, 4...>]
Length: 2, dtype: geometry

In [7]:
california["INTPTLON"].unique()

array(['-119.5434183'], dtype=object)

In [8]:
Egypt["geometry"].unique()

<GeometryArray>
[<MULTIPOLYGON (((34.003 26.732, 34.004 26.708, 33.99 26.72, 33.983 26.734, 3...>]
Length: 1, dtype: geometry

In [9]:
print(Egypt.columns)

Index(['ADMIN', 'ISO_A3', 'ISO_A2', 'geometry'], dtype='object')


In [12]:
target_countries

,ADMIN,ISO_A3,ISO_A2,geometry
68,Egypt,EGY,EG,"MULTIPOLYGON (((34.00294 26.73163, 34.00359 26..."
109,Iraq,IRQ,IQ,"MULTIPOLYGON (((42.89674 37.32491, 42.93705 37..."


In [14]:
print(Egypt.columns)

Index(['ADMIN', 'ISO_A3', 'ISO_A2', 'geometry'], dtype='object')


In [11]:
# Convert speeds to Mbps for easier reading
Egypt['avg_d_mbps'] = Egypt['avg_d_kbps'] / 1000
Egypt['avg_u_mbps'] = Egypt['avg_u_kbps'] / 1000

Iraq['avg_d_mbps'] = Iraq['avg_d_kbps'] / 1000
Iraq['avg_u_mbps'] = Iraq['avg_u_kbps'] / 1000

california['avg_d_mbps'] = california['avg_d_kbps'] / 1000
california['avg_u_mbps'] = california['avg_u_kbps'] / 1000

KeyError: 'avg_d_kbps'

In [15]:
Egypt.to_csv('ookla_Egypt.csv', index=False)

In [ ]:
Iraq.to_csv('ookla_Iraq.csv', index=False)

In [ ]:
california.to_csv('ookla_California.csv', index=False)

In [ ]:
tiles_in_target_regions.to_csv('ookla_Data.csv', index=False)

In [ ]:
# Combine all target regions (Egypt, Iraq, California)
combined_regions = gp.GeoDataFrame(
    pd.concat([target_countries, california], ignore_index=True),
    crs=target_countries.crs
)

# Perform spatial join: Tiles within Egypt, Iraq, and California
tiles_in_target_regions = gp.sjoin(tiles, combined_regions, how="inner", predicate='intersects')



In [ ]:
print(tiles_in_target_regions.columns)

Index(['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices',
       'geometry', 'index_right', 'ADMIN', 'ISO_A3', 'ISO_A2', 'REGION',
       'DIVISION', 'STATEFP', 'STATENS', 'GEOID', 'STUSPS', 'NAME', 'LSAD',
       'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON'],
      dtype='object')


In [ ]:
# Convert speeds to Mbps for easier reading
tiles_in_target_regions['avg_d_mbps'] = tiles_in_target_regions['avg_d_kbps'] / 1000
tiles_in_target_regions['avg_u_mbps'] = tiles_in_target_regions['avg_u_kbps'] / 1000

In [ ]:
print(tiles_in_target_regions.columns)

Index(['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices',
       'geometry', 'index_right', 'ADMIN', 'ISO_A3', 'ISO_A2', 'REGION',
       'DIVISION', 'STATEFP', 'STATENS', 'GEOID', 'STUSPS', 'NAME', 'LSAD',
       'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON',
       'avg_d_mbps', 'avg_u_mbps'],
      dtype='object')


In [ ]:
tiles_in_target_regions.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,index_right,ADMIN,ISO_A3,...,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,avg_d_mbps,avg_u_mbps
57977,0212233310103010,266768,11198,12,1,1,"POLYGON ((-124.22241 42.00033, -124.21692 42.0...",2,NaN,NaN,...,California,00,G4000,A,4.036601e+11,2.030545e+10,+37.1551773,-119.5434183,266.768,11.198
57978,0212233310103102,224953,10423,10,4,4,"POLYGON ((-124.21143 41.99624, -124.20593 41.9...",2,NaN,NaN,...,California,00,G4000,A,4.036601e+11,2.030545e+10,+37.1551773,-119.5434183,224.953,10.423
57979,0212233310103301,176962,11346,13,1,1,"POLYGON ((-124.20593 41.98399, -124.20044 41.9...",2,NaN,NaN,...,California,00,G4000,A,4.036601e+11,2.030545e+10,+37.1551773,-119.5434183,176.962,11.346
57980,0212233310103321,286970,12387,10,4,3,"POLYGON ((-124.20593 41.97583, -124.20044 41.9...",2,NaN,NaN,...,California,00,G4000,A,4.036601e+11,2.030545e+10,+37.1551773,-119.5434183,286.970,12.387
57981,0212233310103323,240103,9922,7,2,2,"POLYGON ((-124.20593 41.97174, -124.20044 41.9...",2,NaN,NaN,...,California,00,G4000,A,4.036601e+11,2.030545e+10,+37.1551773,-119.5434183,240.103,9.922


In [ ]:
column_data = tiles_in_target_regions['avg_d_mbps']
min_value = column_data.min()
print(min_value)

0.001


In [ ]:
column_data = tiles_in_target_regions['avg_d_mbps']
max_value = column_data.max()
print(max_value)

1755.377


In [ ]:
tiles_in_target_regions["avg_u_mbps"]

,avg_u_mbps
57977,11.198
57978,10.423
57979,11.346
57980,12.387
57981,9.922
...,...
4564520,3.859
4564521,4.676
4564522,2.431
4564523,29.400


In [ ]:
tiles_in_target_regions['ADMIN'] = tiles_in_target_regions['ADMIN'].fillna(tiles_in_target_regions['NAME'])
tiles_in_target_regions['NAME'] = tiles_in_target_regions['NAME'].fillna(tiles_in_target_regions['ADMIN'])
tiles_in_target_regions['ISO_A3'] = tiles_in_target_regions['ISO_A3'].fillna("CA")
tiles_in_target_regions.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,index_right,ADMIN,ISO_A3,...,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,avg_d_mbps,avg_u_mbps
57977,0212233310103010,266768,11198,12,1,1,"POLYGON ((-124.22241 42.00033, -124.21692 42.0...",2,California,CA,...,California,00,G4000,A,4.036601e+11,2.030545e+10,+37.1551773,-119.5434183,266.768,11.198
57978,0212233310103102,224953,10423,10,4,4,"POLYGON ((-124.21143 41.99624, -124.20593 41.9...",2,California,CA,...,California,00,G4000,A,4.036601e+11,2.030545e+10,+37.1551773,-119.5434183,224.953,10.423
57979,0212233310103301,176962,11346,13,1,1,"POLYGON ((-124.20593 41.98399, -124.20044 41.9...",2,California,CA,...,California,00,G4000,A,4.036601e+11,2.030545e+10,+37.1551773,-119.5434183,176.962,11.346
57980,0212233310103321,286970,12387,10,4,3,"POLYGON ((-124.20593 41.97583, -124.20044 41.9...",2,California,CA,...,California,00,G4000,A,4.036601e+11,2.030545e+10,+37.1551773,-119.5434183,286.970,12.387
57981,0212233310103323,240103,9922,7,2,2,"POLYGON ((-124.20593 41.97174, -124.20044 41.9...",2,California,CA,...,California,00,G4000,A,4.036601e+11,2.030545e+10,+37.1551773,-119.5434183,240.103,9.922


In [ ]:
tiles_in_target_regions = tiles_in_target_regions.reset_index()
# Rename the 'index' column to 'row_id' (or any name you prefer) if needed
tiles_in_target_regions = tiles_in_target_regions.rename(columns={'index': 'row_id'})

In [ ]:
tiles_in_target_regions.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,index_right,ADMIN,ISO_A3,...,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,avg_d_mbps,avg_u_mbps
57977,0212233310103010,266768,11198,12,1,1,"POLYGON ((-124.22241 42.00033, -124.21692 42.0...",2,California,NaN,...,California,00,G4000,A,4.036601e+11,2.030545e+10,+37.1551773,-119.5434183,266.768,11.198
57978,0212233310103102,224953,10423,10,4,4,"POLYGON ((-124.21143 41.99624, -124.20593 41.9...",2,California,NaN,...,California,00,G4000,A,4.036601e+11,2.030545e+10,+37.1551773,-119.5434183,224.953,10.423
57979,0212233310103301,176962,11346,13,1,1,"POLYGON ((-124.20593 41.98399, -124.20044 41.9...",2,California,NaN,...,California,00,G4000,A,4.036601e+11,2.030545e+10,+37.1551773,-119.5434183,176.962,11.346
57980,0212233310103321,286970,12387,10,4,3,"POLYGON ((-124.20593 41.97583, -124.20044 41.9...",2,California,NaN,...,California,00,G4000,A,4.036601e+11,2.030545e+10,+37.1551773,-119.5434183,286.970,12.387
57981,0212233310103323,240103,9922,7,2,2,"POLYGON ((-124.20593 41.97174, -124.20044 41.9...",2,California,NaN,...,California,00,G4000,A,4.036601e+11,2.030545e+10,+37.1551773,-119.5434183,240.103,9.922


In [ ]:
tiles_in_target_regions.tail()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,index_right,ADMIN,ISO_A3,...,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,avg_d_mbps,avg_u_mbps
4564520,1230201221310131,932,3859,169,6,4,"POLYGON ((48.46619 29.98349, 48.47168 29.98349...",1,Iraq,IRQ,...,Iraq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.932,3.859
4564521,1230201221310311,10820,4676,48,2,2,"POLYGON ((48.46619 29.97397, 48.47168 29.97397...",1,Iraq,IRQ,...,Iraq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.820,4.676
4564522,1230201221311203,673,2431,384,6,1,"POLYGON ((48.47717 29.96921, 48.48267 29.96921...",1,Iraq,IRQ,...,Iraq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.673,2.431
4564523,1230201221311212,1141,29400,203,3,1,"POLYGON ((48.48267 29.96921, 48.48816 29.96921...",1,Iraq,IRQ,...,Iraq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.141,29.400
4564524,1230201221311221,4665,1596,19,1,1,"POLYGON ((48.47717 29.96445, 48.48267 29.96445...",1,Iraq,IRQ,...,Iraq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.665,1.596


In [ ]:
##################################################################### WEIRD STUFF BELOW ##########################################################

In [ ]:
# Load VIIRS radiance data
viirs_url = "path_or_api_to_viirs_data"  # Replace with actual source
viirs_data = gp.read_file(viirs_url)

# Calculate daily change in radiance
viirs_data['daily_change'] = viirs_data.groupby('region_id')['radiance'].diff()

# Define significant drop threshold (e.g., -20%)
DROP_THRESHOLD = -0.2
viirs_data['significant_drop'] = viirs_data['daily_change'] < DROP_THRESHOLD


In [ ]:
# Aggregate Ookla data daily
ookla_daily = tiles_in_target_regions.groupby(['NAME', 'row_id']).agg(
    avg_d_mbps=('avg_d_mbps', 'mean'),
    avg_u_mbps=('avg_u_mbps', 'mean'),
    tests=('tests', 'sum')
).reset_index()

# Calculate daily changes in speeds
ookla_daily['daily_change_d'] = ookla_daily.groupby('NAME')['avg_d_mbps'].diff()
ookla_daily['daily_change_u'] = ookla_daily.groupby('NAME')['avg_u_mbps'].diff()

# Define significant drop threshold for internet performance (e.g., -30%)
INTERNET_DROP_THRESHOLD = -0.3
ookla_daily['significant_drop'] = (
    (ookla_daily['daily_change_d'] < INTERNET_DROP_THRESHOLD) |
    (ookla_daily['daily_change_u'] < INTERNET_DROP_THRESHOLD)
)


In [ ]:
ookla_daily.head()

,NAME,row_id,avg_d_mbps,avg_u_mbps,tests,daily_change_d,daily_change_u,significant_drop
0,California,52094,556.668,11.065,1,NaN,NaN,False
1,California,52095,71.942,9.605,2,-484.726,-1.460,True
2,California,52096,332.837,22.510,1,260.895,12.905,False
3,California,52097,280.202,15.500,4,-52.635,-7.010,True
4,California,52098,332.443,14.074,3,52.241,-1.426,True


In [ ]:
ookla_daily.tail()

,NAME,row_id,avg_d_mbps,avg_u_mbps,tests,daily_change_d,daily_change_u,significant_drop
142942,Iraq,4419352,20.026,20.241,1,9.511,12.208,False
142943,Iraq,4419353,7.838,4.620,10,-12.188,-15.621,True
142944,Iraq,4419354,3.752,12.405,1,-4.086,7.785,True
142945,Iraq,4419520,28.196,10.531,6,24.444,-1.874,True
142946,Iraq,4419521,81.509,17.116,74,53.313,6.585,False


In [ ]:
# Ensure spatial and temporal alignment
viirs_data = viirs_data.to_crs(tiles_in_target_regions.crs)

# Merge VIIRS and Ookla data on region and date
combined_data = pd.merge(
    viirs_data[['region_id', 'date', 'significant_drop']],
    ookla_daily[['region_id', 'date', 'significant_drop']],
    on=['region_id', 'date'],
    suffixes=('_viirs', '_ookla')
)

# Composite metric: Outage detected if both datasets show significant drops
combined_data['outage_detected'] = (
    combined_data['significant_drop_viirs'] & combined_data['significant_drop_ookla']
)


In [ ]:
# Summary of outages
outage_summary = combined_data[combined_data['outage_detected']].groupby('region_id').agg(
    outage_days=('date', 'count')
).reset_index()

# Visualization
fig, ax = plt.subplots(figsize=(12, 8))
combined_data.plot(
    column='outage_detected', cmap='coolwarm', legend=True, ax=ax
)
plt.title("Detected Outages (VIIRS + Ookla)")
plt.show()
